<a href="https://colab.research.google.com/github/Shirleyhywl/hello-world/blob/master/Python2_7Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ADD OTHER IMPORTS HERE AS NECESSARY
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import sys
import csv
import os

# SK-learn libraries for learning.
#from sklearn.pipeline import Pipeline
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import GridSearchCV
# from sklearn.datasets import fetch_openml

# SK-learn libraries for evaluation.
#from sklearn.metrics import confusion_matrix
#from sklearn import metrics
#from sklearn.metrics import classification_report

# Theano imports
import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs
np.random.seed(0) # Setting random seed

cpu
float64


In [13]:
print("python version =",sys.version)
print("Pandas Version = ",pd.__version__)
print("Numpy Version = ",np.__version__)
print(theano.__version__)

('python version =', '2.7.15+ (default, Nov 27 2018, 23:36:35) \n[GCC 7.3.0]')
('Pandas Version = ', u'0.24.2')
('Numpy Version = ', '1.16.2')
0.8.2


In [9]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
os.getcwd()

'/content'

### 5. Data Import and Cleaning
 - There is a Train and Test sample in the data from Kaggle
 - We should split the Train data into Train and Dev - and optimize on Dev; Save Test for the very last step in each method.
 - Select an appropriate scoring method (per the Kaggle contest and maybe one other as well)

###Train Data (training.csv)

*  There is total 31 columns (features) and 7049 rows (images)
  
    *  30 columns for 15 predicted keypoint pair specified by a (x,y) real-value pair in the space of pixel indices
    * 1 column for 96*96 pixel input image consists a list of pixels (ordered by row), as integers in (0,255) in string type



In [11]:
def Data_PreProcess(train = True):
  """ Loads data based on the purpose of training or testing (train=True, test=FALSE).
  Tranform last column of imagine information from string to numpy array.
  Finally, normalized keypoint pairs based on each keypoint average."""
  
  # load data 
  if train: 
    data = pd.read_csv('drive/My Drive/FinalProject_w207/training.csv')
    df = pd.DataFrame(data)
  else:
    data = pd.read_csv('drive/My Drive/FinalProject_w207/test.csv')
    df = pd.DataFrame(data)
  print('There is %i columns (features)' % df.shape[0])
  print('There is %i rows (images)' % df.shape[1])
  print(df.count())
  
  # remove the rows which contain NA value
  df = df.dropna()
  print('There is %i rows (images) after remove NA values' % df.shape[0])
  
  # transform image data and load as X value
  df['Image']=df['Image'].apply(lambda X: np.fromstring(X, sep=' ')) # change string in 'image' column to nparray
  image= np.vstack(df['Image'].to_numpy())   # Convert 'image'column to nparray
  X = image/255
  
  
  # Normalize keypoint pairs and load as Y value
  keypoints = df[list(df.columns)[:-1]].to_numpy()
  mean = df[list(df.columns)[:-1]].mean(axis=1).to_numpy()
  mean_array = np.repeat(mean.reshape(df.shape[0],1),30,axis =1)
  Y= (keypoints-mean_array)/mean_array
  
  return X,Y

X,Y = Data_PreProcess()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
    X.shape, X.min(), X.max()))
print("Y.shape == {}; Y.min == {:.3f}; Y.max == {:.3f}".format(
    Y.shape, Y.min(), Y.max()))

There is 7049 columns (features)
There is 31 rows (images)
left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y 

In [8]:
!pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
!pip install https://github.com/Lasagne/Lasagne/archive/master.zip
!pip install -r https://raw.githubusercontent.com/dnouri/kfkd-tutorial/master/requirements.txt
!pip install git+https://github.com/Lasagne/Lasagne
!pip install nolearn
!pip install scikit-learn==0.18.2
!pip install theano==0.8.2

SyntaxError: ignored

In [5]:
pip install theano==0.8.2

In [12]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, 9216),  # 96x96 input pixels per batch, None translates to a variable batch size
    hidden_num_units=100,  # number of units in hidden layer
    output_nonlinearity=None,  # activation function is Rectifier
    output_num_units=30,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,  # flag to indicate we're dealing with regression problem
    max_epochs=400,  # we want to train this many epochs
    verbose=1, # print out information during training
    )

X, Y = Data_PreProcess()
net1.fit(X, Y)

There is 7049 columns (features)
There is 31 rows (images)
left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y 

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: DimShuffle{x,x}(TensorConstant{1})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/opt.py", line 1772, in process_node
    replacements = lopt.transform(node)
  File "/usr/local/lib/python2.7/dist-packages/theano/tensor/opt.py", line 5825, in constant_folding
    no_recycling=[])
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/op.py", line 970, in make_thunk
    no_recycling)
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/op.py", line 879, in make_c_thunk
    output_storage=node_output_storage)
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/cc.py", line 1200, in make_thunk
    keep_lock=keep_lock)
  File "/usr/local/lib/python2.7/dist-packages/theano/gof/cc.py", line 1143, in __compile__
    keep_lock=keep_lock)
  File "/usr/local/lib/python2.7/dist-pack

AttributeError: ignored